# Speaker recognition using singing or speaking voice

This project aims at recognizing speakers through a joint representation learning of singing and speaking voices from them. 

In [1]:
import pandas as pd
import numpy as np
import torch
import json

### JukeBox dataset analysis

[Voxceleb Link](https://mm.kaist.ac.kr/datasets/voxceleb/#downloads)

In [2]:
metadata = pd.read_csv("./jukebox_metadata.csv")

#creating artist id and name pairs
idname_df = metadata[['artist_id', 'artist_name']]
ids = list(idname_df['artist_id'])
names = list(idname_df['artist_name'])

id_name_dict = {}

for i in range(0, len(ids)):
    
    if ids[i] in id_name_dict:
        continue
    else:
        id_name_dict[int(ids[i])] = names[i]

with open("./jukebox_id_artist.json", "w+") as data:
    json.dump(id_name_dict, data)

### Jukebox and VoxCeleb common artists

In [3]:
#jukebox artists
with open("./jukebox_id_artist.json", 'r') as juke:
    jukebox = json.load(juke)

jukebox_arts = [j for i,j in jukebox.items()]
jukebox_arts = [i.replace(" ", "_").lower() for i in jukebox_arts]
print(jukebox_arts[:10])


#voxceleb artists
voxDf = pd.read_csv("./vox1_meta.csv", delimiter="\t")
vox_arts = list(voxDf["VGGFace1 ID"])
vox_arts = [k.lower() for k in vox_arts]
print(vox_arts[:10])

#commons
commons = list(set(jukebox_arts).intersection(set(vox_arts)))
with open("commons_in_vox_and_jukebox.txt", 'w+') as commonData:

    for artist in commons:
        commonData.write(artist)
        commonData.write("\n")



['a-lin', 'adira', 'adrian_minune', 'afgan', 'ai', 'aiko', 'aimer', 'albert_west', 'alex_mica', 'alexandra_stan']
['a.j._buckley', 'a.r._rahman', 'aamir_khan', 'aaron_tveit', 'aaron_yoo', 'abbie_cornish', 'abigail_breslin', 'abigail_spencer', 'adam_beach', 'adam_brody']


### housekeeping for arranging vox and juke data

In [4]:
jukemeta = pd.read_csv("jukebox_metadata.csv")
voxmeta = pd.read_csv("vox1_meta.csv", delimiter='\t')
voxmeta['VGGFace1 ID'] = voxmeta['VGGFace1 ID'].str.casefold()
jukemeta['artist_name'] = jukemeta['artist_name'].str.casefold()
jukemeta['artist_name'] = jukemeta['artist_name'].apply(lambda x: x.replace(" ", "_"))

In [20]:
### creating dictionary for voxid:artist pair

voxcommon = voxmeta[voxmeta['VGGFace1 ID'].isin(commons)]
voxid_celeb = {}
for i in range(0, len(voxcommon)):
    voxid_celeb[voxcommon.iloc[i]['VoxCeleb1 ID']]=voxcommon.iloc[i]['VGGFace1 ID']

with open("./vox_id_celeb.json", 'w+') as voxidartist:
    
    json.dump(voxid_celeb, voxidartist)

In [40]:
### creating dictionary for jukeID:artist pair
jukecommon = jukemeta[jukemeta['artist_name'].isin(commons)]
#print(list(jukecommon))

jukeid_celeb = {}
for c in commons:
    jukeid = str(jukecommon[jukecommon['artist_name'] == c]['artist_id'].unique()[0])
    jukeid_celeb[jukeid] = c

with open("./juke_id_celeb.json", 'w+') as j



{'1162': 'chris_martin',
 '475': 'cher',
 '1128': 'troye_sivan',
 '1057': 'stevie_wonder',
 '372': 'anne_hathaway',
 '649': 'jack_black',
 '789': 'lea_michele',
 '75': 'charlotte_gainsbourg',
 '924': 'nicole_scherzinger',
 '466': 'carrie_underwood',
 '867': 'meat_loaf',
 '814': 'lorde',
 '727': 'josh_groban',
 '682': 'jill_scott',
 '776': 'kylie_minogue',
 '663': 'janet_jackson',
 '1046': 'smokey_robinson',
 '914': 'nelly_furtado',
 '420': 'blake_shelton',
 '503': 'cyndi_lauper',
 '990': 'rita_ora',
 '680': 'jessie_j',
 '761': 'kenny_rogers',
 '892': 'miley_cyrus',
 '1020': 'sammy_davis_jr.',
 '452': 'bruno_mars',
 '842': 'marie_osmond',
 '359': 'amanda_seyfried',
 '790': 'lea_salonga'}

In [24]:
jukecommon[:10]

,Unnamed: 0,song_name,artist_id,artist_name,partition,language_label,gender_label,full_path
2411,2411,0_1.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_1.wav
2412,2412,0_2.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_2.wav
2413,2413,0_3.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_3.wav
2414,2414,0_4.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_4.wav
2415,2415,0_5.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_5.wav
14825,14825,0_1.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_1.wav
14826,14826,0_2.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_2.wav
14827,14827,0_3.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_3.wav
14828,14828,0_4.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_4.wav
14829,14829,0_5.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_5.wav
